In [1]:
import neuroglancer
import neuroglancer.webdriver
from neuroglancer.server import global_server_args
from ngtracts.tracts import TractSource


In [6]:
# Orientation color coding following  Pajevic & Pierpaoli, MRM (1999)
# We follow the mirror + rotational symmetry convention
colormap_orient = """
vec3 colormapOrient(vec3 orient) {
  vec3 result;
  result.r = abs(orient[0]);
  result.g = abs(orient[1]);
  result.b = abs(orient[2]);
  return clamp(result, 0.0, 1.0);
}
"""

# Runing this in the skeleton shader uses orientation to color tracts
orient_shader = colormap_orient + """
#uicontrol bool orient_color checkbox(default=false)
void main() {
  if (orient_color)
    emitRGB(colormapOrient(orientation));
  else
  	emitDefault();
}
"""

mri_shader = """
#uicontrol float brightness slider(min=0.0, max=1.0, default=0.5)
void main() {
    float x = clamp(toNormalized(getDataValue()) * brightness, 0.0, 1.0);
    float angle = 2.0 * 3.1415926 * (4.0 / 3.0 + x);
    float amp = x * (1.0 - x) / 2.0;
    vec3 result;
    float cosangle = cos(angle);
    float sinangle = sin(angle);
    result.r = -0.14861 * cosangle + 1.78277 * sinangle;
    result.g = -0.29227 * cosangle + -0.90649 * sinangle;
    result.b = 1.97294 * cosangle;
    result = clamp(x + amp * result, 0.0, 1.0);
    emitRGB(result);
}
"""


In [3]:

global_server_args['bind_port'] = '9999'
viewer = neuroglancer.Viewer(token='1')
print(viewer.get_viewer_url())


http://127.0.0.1:9999/v/1/


In [7]:
TRK = "https://dandiarchive.s3.amazonaws.com/blobs/d4a/c43/d4ac43bd-6896-4adf-a911-82edbea21f67"
NII = "https://dandiarchive.s3.amazonaws.com/blobs/3de/a2d/3dea2d82-8af8-434f-b7a9-60a21d891985"

with viewer.txn() as state:
    state.layers.append(
        name="mri",
        layer=neuroglancer.ImageLayer(
            source=[
                "nifti://" + NII
            ],
            shader=mri_shader,
        )
    )
    state.layers.append(
        name="tracts",
        layer=neuroglancer.SegmentationLayer(
            source=[
                TractSource(TRK),
            ],
            skeleton_shader=orient_shader,
            selected_alpha=0,
            not_selected_alpha=0,
            segments=[1],
        ),
    )
